In [ ]:
!pip install pandas==1.3.4

In [ ]:
import pandas as pd
import matplotlib
import IPython


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')


In [ ]:
# 경기도 인구 파일 불러오기

filepath = './'
population = pd.read_csv(filepath + "/주민등록인구집계현황202409.csv", encoding='euc-kr')
population.head(2)

In [ ]:
print("데이터 크기 :", population.shape)

In [ ]:
#결측치 확인
population.isnull().sum()

In [7]:
# 경험상 젊은 여성들이 많은 곳이 활기 넘치는 곳이었습니다. 맘카페 등등
popula_female = population.iloc[:,-12:]
popula_info = population.iloc[:,:4]
pop_female_info = pd.concat([popula_info, popula_female], axis=1)

In [ ]:
pop_female_info.head(3)

In [ ]:
# 2022년은 분석에서 제외
pop_female_info['연도'].value_counts().sort_index()

In [10]:
pop_female_info = pop_female_info[pop_female_info['연도']!=2022]

In [11]:
pop_female_info.rename(columns={"0~9세 (여)":"10대_미만",
                                "10~19세 (여)":"10대",
                                "20~29세 (여)":"20대",
                                "30~39세 (여)":"30대",
                                "40~49세 (여)":"40대",
                                "50~59세 (여)":"50대",
                                "60~69세 (여)":"60대"
                                }, inplace=True)

In [12]:
pop_female_info['70대_이상']=pop_female_info["70~79세 (여)"]+pop_female_info["80~89세 (여)"]+\
pop_female_info["90~99세 (여)"] + pop_female_info["100세 이상 (여)"]

In [ ]:
pop_female_info.info()

In [14]:
df_female = pop_female_info.drop(['70~79세 (여)', '80~89세 (여)', '90~99세 (여)','100세 이상 (여)'], axis=1)

In [ ]:
df_female.head(2)

In [ ]:
"""행정구역이 동으로 분류되는 통계만 찾아보자"""
df_female_dong=df_female[df_female['행정구역명'].str.contains('동')]
df_female_dong.head()

In [ ]:
df_female_dong.shape

In [ ]:
# 40대 이하 비율 (%)과 50대 이상 비율(%) 구하기
under40s = df_female_dong['10대_미만']+df_female_dong['10대']+df_female_dong['20대']+df_female_dong['30대']+df_female_dong['40대']
over50s = df_female_dong['50대']+df_female_dong['60대']+df_female_dong['70대_이상']

under40_percent = round((under40s / df_female_dong['총 인구수 (여)']) * 100, 2)
over50_percent = round((over50s / df_female_dong['총 인구수 (여)']) * 100, 2)

df_female_dong['40대_이하_여성비율(%)'] = under40_percent
df_female_dong['50대_이상_여성비율(%)'] = over50_percent

df_female_dong.head(2)

In [ ]:
# split() 설명
x = "경기도 고양시 덕양구 고양동"
print("split() 적용 전 :",x)
print("split() 적용 후 :",x.split())
print("동 추출 결과 :",x.split()[-1])
print("시 추출 결과 :",x.split()[1])


In [ ]:
df_female_dong['동'] = df_female_dong['행정구역명'].apply(lambda x:x.split()[-1])
df_female_dong['시'] = df_female_dong['행정구역명'].apply(lambda x:x.split()[1])

In [ ]:
df_female_dong.sample()

In [ ]:
df_female_dong['시-동'] = df_female_dong['시']+ " " +df_female_dong['동']

In [23]:
df_result = df_female_dong[['연도','시','동','시-동','총 인구수 (여)','40대_이하_여성비율(%)', '50대_이상_여성비율(%)']]

In [ ]:
df_result.isnull().sum()

In [ ]:
# 결측치를 눈으로 확인하는 단계
df_result[df_result['40대_이하_여성비율(%)'].isnull()]

In [ ]:
df_result.dropna(inplace=True)

In [27]:
# 최근 5년 동안 대상 (2016 ~ 2021년)
df_resent = df_result[df_result['연도']>2015]

In [ ]:
df_result.sample(3)

In [ ]:
df_resent.sample()

In [ ]:
df_resent_group = df_resent.groupby(['시-동'])['40대_이하_여성비율(%)', "50대_이상_여성비율(%)"].mean()

In [31]:
df_resent_group.reset_index(inplace=True)

In [ ]:
df_resent_group.sort_values(by='40대_이하_여성비율(%)', ascending=False)[:20]

In [33]:
df_top20 = df_resent_group.sort_values(by='40대_이하_여성비율(%)', ascending=False)[:20]

In [34]:
x = df_top20['시-동']
y = df_top20['40대_이하_여성비율(%)']

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("2016 ~ 2021년, 40대 이하 여성이\n가장 많이 사는 지역 상위 20개 동", fontsize=15)
plt.bar(x, y, alpha=0.7, color='darkgreen')
plt.xticks(size=12, rotation=45)
plt.yticks(size=12)
plt.ylim(70,90)
plt.ylabel('%', size=20)
plt.grid(True, alpha=0.2)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("2016 ~ 2021년, 40대 이하 여성이\n가장 많이 사는 지역 상위 20개 동", fontsize=20)
plt.barh(x, y, alpha=0.7, color='darkgreen')
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlim(50,90)
plt.grid(True, alpha=0.2)